# Importing

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import ee
import geemap
import fiona
import time
import json
from pathlib import Path
import datetime
import numpy as np
#from utils.gee import export_sar, export_opt, export_maskcloud, export_opt_l1
import yaml
from utils.gee import export_nv, export_sar

# Authentication

In [3]:
ee.Authenticate()
ee.Initialize()


Successfully saved authorization token.


In [4]:
#roi = ee.Geometry.Rectangle([cfg['xmin'], cfg['ymin'], cfg['xmax'], cfg['ymax']])
roi = ee.Geometry.Rectangle([-75.9909722200000033, -16.0477750000000015, -42.9509722199999970,7.2722249999999997])


In [5]:
map_view = geemap.Map(center=(-5, -58), zoom=4)
map_view

Map(center=[-5, -58], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(To…

In [6]:
map_view.addLayer(roi, {}, 'Location')

In [7]:
opt_vizParams = {
  'bands': ['B4', 'B3', 'B2'],
  'min': 0,
  'max': 2048
}

sar_vizParamsVV = {
  'bands': ['VV'],
  'min': 0,
  'max': 1
}

sar_vizParamsHH = {
  'bands': ['HH'],
  'min': 0,
  'max': 1
}

cloud_vizParams = {
  'bands': ['MSK_CLDPRB'],
  'min': 0,
  'max': 100
}

nv_vizParams = {
  'bands': ['avg_rad'],
  'min': 0,
  'max': 100
}

In [8]:
nv_collection = ee.ImageCollection('NOAA/VIIRS/DNB/MONTHLY_V1/VCMCFG').filterDate('2023-01-01', '2023-12-31').filterBounds(roi)

In [9]:
im = nv_collection.first().clip(roi)
map_view.addLayer(im, nv_vizParams, f'nv')

In [10]:
export_nv(im, roi, 'nv')

In [32]:
sar_collection = ee.ImageCollection('JAXA/ALOS/PALSAR-2/Level2_2/ScanSAR').filterDate('2022-01-15', '2022-01-30').filterBounds(roi)
sar_collection = sar_collection.filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
sar_im = sar_collection.mean()
map_view.addLayer(sar_im, sar_vizParamsVV, f'sar')

In [31]:
export_sar(sar_im, roi, 'sar_amz_VV_2', 500)